# Wiedemann

In [2]:
def suite_kyrlov(A,v):
    # on pourrait améliorer le range pour calculer moins de colonnes..
    n = A.dimensions()[0]
    Avt = A*v
    B = v
    Q = Avt
    for i in range(ceil(log(n,2))):
        A = A^2
        B = block_matrix(1,2,[B,Q])
        Q = A*B
    return block_matrix(1,2,[B,Q])

In [3]:
def Pade_approx(A,B,k):
    R0 = A
    U0 = 1
    V0 = 0
    R1 = B
    U1 = 0
    V1 = 1
    while (R0.degree() >= k):
        Q1 = R0//R1
        R0,R1 = R1, R0%R1
        U0,U1 = U1, U0 - Q1*U1
        V0,V1 = V1, V0 - Q1*V1
    if (gcd (A,V1) == 1):
        return (R0//V0(0),V0//V0(0))
    return 0

In [4]:
def Berlekamp_Massey(A):
    K = parent(A[0])
    pR.<x> = K[]
    Pa = pR(A)
    N = Pa.degree()
    R,V = Pade_approx(x^N,Pa,floor(N/2))
    return V.reverse()

In [5]:
def Wiedemann(A):
    n = A.dimensions()[0]
    K = A.base_ring()
    V = K^n
    u = matrix(V.random_element())
    r = matrix(V.random_element()).transpose()
    R = suite_kyrlov(A,r)
    N = [(u*ri)[0] for ri in R.transpose()[:2*n+1]]
    return Berlekamp_Massey(N)

In [6]:
K = GF(next_prime(300))
D = 10

M = MatrixSpace(K,D)

A = M.random_element()

In [7]:
muA = A.minimal_polynomial()

In [8]:
P = Wiedemann(A)

In [9]:
P == muA

True

In [10]:
muA%P

0

# Block Wiedemann

In [20]:
D = 10
m = 4
M = MatrixSpace(K,D)


In [21]:
A = M.random_element()

### inverse modulo $x^d$

In [22]:
def inv_x_d(M,d):
    parentM = parent(M)
    n = M.dimensions()[0]
    K = M.base_ring().base()
    MS.<y> = PolynomialRing(MatrixSpace(K,n))
    
    
    Mpol = sum([M.coefficient_matrix(i)*y^i for i in range(1+M.degree())])
    pR.<x> = K[]
    l  = Mpol.inverse_series_trunc(d)(x)
    return parentM(l)

## weak Popov form to Popov form

In [23]:
def monic_pivot(M):
    n = M.dimensions()[0]
    lead_pos = M.leading_positions()
    return Matrix([(M[i,lead_pos[i]].lc()^(-1)*M[i]).list() for i in range(n)])

def ascending_transformation(M):
    I = M.leading_positions()
    row_ind = [(i,j,I[i]) for i,j in enumerate(M.row_degrees())]
    
    def pair_order(a):
        return a[1]+a[2]*10^(-1)
    
    return matrix([M[i] for i,j,k in sorted(row_ind,key = pair_order)])

def simple_transformation(M,k,l):
    x = parent(M[0,0]).gen()
    I_k = M.leading_positions()[k]
    while(True):
        P_k = M[k,I_k]
        mon = M[l,I_k]
        deg_P,deg_m = P_k.degree(), mon.degree()
        if (deg_P > deg_m):
            return M
        coef_P,coef_m = P_k.lc(), mon.lc()

        c = coef_m * coef_P^(-1)
        e = deg_m-deg_P
        M[l] = M[l] - c*x^e*M[k]

def popov_sage(M):
    I = M.leading_positions()
    row_ind = [(i,j) for i,j in enumerate(I)]
    
    def pair_order(a):
        return a[1]
    
    return matrix([M[i] for i,j in sorted(row_ind,key = pair_order)])        

def weak_to_popov(M):
    if not(M.is_weak_popov()):
        M = M.weak_popov_form()
    A = ascending_transformation(M)
    n = A.dimensions()[0]
    I = A.leading_positions()
    D = [A[i,j].degree() for i,j in enumerate(I)]
    
    for k in range(n):
        if (I[k] != -1):
            for l in range(k+1,n):
                    if (A[l,I[k]].degree() - D[k] >= 0):
                        A = simple_transformation(A,k,l)
    return popov_sage(monic_pivot(A))

In [30]:
def Block_Wiedemann(A):
    D = A.dimensions()[0]
    K = A.base_ring()
    Mvect = MatrixSpace(K,m,D)
    
    U = Mvect.random_element()
    V = Mvect.random_element().transpose()
    
    R = suite_kyrlov(A,V)
    P = ceil(2*D/m)
    print(R.dimensions(), P)
    S = [U*R[:,k:k+m] for k in range(0,m*P,m)]
    pR.<x> = K[]
    S = sum([S[i]*x^i for i in range(len(S))])

    Mat = block_matrix(2,2,[[1,S.reverse(degree=P-1)],[0,x^P]])
    return Mat,S.reverse(degree = P-1),P
    popov_Mat = Mat.weak_popov_form(ordered = True)
    print(popov_Mat.degree_matrix())
    print(popov_Mat.leading_positions())
    L,R = popov_Mat[:m,:m], popov_Mat[m:,m:]
    
    Linv = inv_x_d(L,P)
    return L,Linv, R,S

In [31]:
Mat,S,P =  Block_Wiedemann(A)

(10, 128) 5


In [50]:
G = block_matrix([[S],[-1]])

In [57]:
A = G.minimal_approximant_basis(P)

In [42]:
L.determinant().monic()

x^30 + 292*x^29 + x^28 + 73*x^27 + 114*x^26 + 192*x^25 + 147*x^24 + 26*x^23 + 201*x^22 + 250*x^21 + 266*x^20 + 168*x^19 + 171*x^18 + 134*x^17 + 71*x^16 + 301*x^15 + 133*x^14 + 197*x^13 + 267*x^12 + 218*x^11 + 282*x^10 + 287*x^9 + 302*x^8 + 241*x^7 + 275*x^6 + 282*x^5 + 114*x^4 + 124*x^3 + 110*x^2 + 174*x + 196

In [59]:
A[:3,:3].determinant()

x^30 + 292*x^29 + x^28 + 73*x^27 + 114*x^26 + 192*x^25 + 147*x^24 + 26*x^23 + 201*x^22 + 250*x^21 + 266*x^20 + 168*x^19 + 171*x^18 + 134*x^17 + 71*x^16 + 301*x^15 + 133*x^14 + 197*x^13 + 267*x^12 + 218*x^11 + 282*x^10 + 287*x^9 + 302*x^8 + 241*x^7 + 275*x^6 + 282*x^5 + 114*x^4 + 124*x^3 + 110*x^2 + 174*x + 196

In [410]:
M.truncate(s+1)

[       247*x^20 + 253*x^19 + 258*x^18 + 24*x^17 + 279*x^16 + 235*x^15 + 24*x^14 + 116*x^13 + 305*x^12 + 276*x^11 + 234*x^10 + 9*x^9 + 44*x^8 + 74*x^7 + 249*x^6 + 61*x^5 + 73*x^4 + 256*x^3 + 92*x^2 + 52*x + 160  131*x^20 + 93*x^19 + 213*x^18 + 156*x^17 + 225*x^16 + 28*x^15 + 44*x^14 + 122*x^13 + 149*x^12 + 108*x^11 + 294*x^10 + 269*x^9 + 286*x^8 + 125*x^7 + 150*x^6 + 49*x^5 + 272*x^4 + 276*x^3 + 249*x^2 + 133*x + 197         140*x^20 + 126*x^19 + 96*x^18 + 303*x^17 + 281*x^16 + 45*x^15 + 25*x^14 + 64*x^13 + 130*x^12 + 89*x^11 + 17*x^10 + 138*x^9 + 173*x^8 + 112*x^7 + 9*x^6 + 287*x^5 + 72*x^4 + 219*x^3 + 9*x^2 + 260*x + 226]
[   296*x^20 + 236*x^19 + 79*x^18 + 170*x^17 + 152*x^16 + 147*x^15 + 279*x^14 + 69*x^13 + 45*x^12 + 127*x^11 + 200*x^10 + 230*x^9 + 237*x^8 + 136*x^7 + 210*x^6 + 8*x^5 + 112*x^4 + 138*x^3 + 34*x^2 + 240*x + 209        234*x^20 + 278*x^19 + 50*x^18 + 92*x^17 + 245*x^16 + 74*x^15 + 290*x^14 + 224*x^13 + 227*x^12 + 197*x^11 + 255*x^10 + 44*x^9 + 13*x^8 + 299*x^7 + 28*x

In [411]:
S

[    13*x^20 + 144*x^19 + 199*x^18 + 274*x^17 + 248*x^16 + 259*x^15 + 52*x^14 + 33*x^13 + 89*x^12 + 69*x^11 + 114*x^10 + 197*x^9 + 245*x^8 + 69*x^7 + 15*x^6 + 304*x^5 + 187*x^4 + 75*x^3 + 175*x^2 + 199*x + 164          14*x^20 + 59*x^19 + 45*x^18 + 70*x^17 + 154*x^16 + 97*x^15 + 294*x^14 + 95*x^13 + 109*x^12 + 203*x^11 + 96*x^10 + 77*x^9 + 303*x^8 + 36*x^7 + 270*x^6 + 20*x^5 + 68*x^4 + 230*x^3 + 9*x^2 + 133*x + 267 209*x^20 + 300*x^19 + 244*x^18 + 277*x^17 + 238*x^16 + 154*x^15 + 129*x^14 + 270*x^13 + 266*x^12 + 268*x^11 + 53*x^10 + 156*x^9 + 251*x^8 + 134*x^7 + 155*x^6 + 15*x^5 + 265*x^4 + 180*x^3 + 53*x^2 + 180*x + 98]
[  40*x^20 + 301*x^19 + 93*x^18 + 249*x^17 + 130*x^16 + 114*x^15 + 154*x^14 + 213*x^13 + 289*x^12 + 211*x^11 + 35*x^10 + 240*x^9 + 123*x^8 + 220*x^7 + 153*x^6 + 66*x^5 + 303*x^4 + 19*x^3 + 122*x^2 + 133*x + 46     87*x^20 + 122*x^19 + 127*x^18 + 160*x^17 + 69*x^16 + 215*x^15 + 90*x^14 + 227*x^13 + 16*x^12 + 106*x^11 + 67*x^10 + 192*x^9 + 115*x^8 + 63*x^7 + 28*x^6 + 142